In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import pandas_gbq
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_race`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


Downloading: 100%|██████████|


In [4]:
GP=df_bq2
GP=GP.fillna(0)
GP["Position"]=GP["Position"].astype(int)
GP["GridPosition"]=GP["GridPosition"].astype(int)
GP["Points"]=GP["Points"].astype(int)
GP.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,30,L LAWSON,LAW,lawson,RB,6692FF,rb,Liam,Lawson,Liam Lawson,None,NZL,9,9,19,NaT,NaT,NaT,0 days 00:01:10.563000,Finished,2,2024,United States Grand Prix
1,22,Y TSUNODA,TSU,tsunoda,RB,6692FF,rb,Yuki,Tsunoda,Yuki Tsunoda,https://media.formula1.com/d_driver_fallback_i...,JPN,14,14,10,NaT,NaT,NaT,NaT,+1 Lap,0,2024,United States Grand Prix
2,27,N HULKENBERG,HUL,hulkenberg,Haas F1 Team,B6BABD,haas,Nico,Hulkenberg,Nico Hulkenberg,https://media.formula1.com/d_driver_fallback_i...,GER,8,8,11,NaT,NaT,NaT,0 days 00:01:02.957000,Finished,4,2024,United States Grand Prix
3,20,K MAGNUSSEN,MAG,kevin_magnussen,Haas F1 Team,B6BABD,haas,Kevin,Magnussen,Kevin Magnussen,https://media.formula1.com/d_driver_fallback_i...,DEN,11,11,8,NaT,NaT,NaT,0 days 00:01:19.782000,Finished,0,2024,United States Grand Prix
4,10,P GASLY,GAS,gasly,Alpine,0093cc,alpine,Pierre,Gasly,Pierre Gasly,https://media.formula1.com/d_driver_fallback_i...,FRA,12,12,6,NaT,NaT,NaT,0 days 00:01:30.558000,Finished,0,2024,United States Grand Prix


In [5]:
#Get the counts by final position in race

In [6]:
grouped = GP.groupby(['FullName','Position'])["GP"].agg('count').reset_index()
grouped.columns=["Driver","Position","Count"]
grouped["Rank"] = grouped.groupby(['Position'])["Count"].rank(method="dense", ascending=False)
grouped=grouped.sort_values(by=["Position","Count"],ascending=[True, False])

In [7]:
#Plot 1 lap at a time

In [8]:
grouped2=grouped[(grouped['Position'].isin([1]))&(grouped['Count']>1)]

In [9]:
position=grouped2["Position"].iloc[0]

In [10]:
fig_lap=px.bar(grouped2, 
           x="Driver", 
           y="Count", 
        #    facet_col='Position', facet_col_wrap=1, 
           height=800,
           width=1400,
           template='simple_white',
            color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 }
        )
fig_lap.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title="Driver All Time Race Wins",
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_lap.update_traces(marker_color='#246DF8')

In [11]:
fig_lap.write_html("/Users/rdesh723/statpulse-html/plots/All Time/Driver Race Wins All Time.html",full_html=False, include_plotlyjs='cdn')

In [12]:
#Plot the top in each final position

In [13]:
grouped_top=grouped[grouped["Rank"]==1]
grouped_top=grouped_top[grouped_top["Position"]!=0]

In [14]:
fig_top=px.bar(grouped_top, 
           x="Position", 
           y="Count", 
           color='Count',
           height=800,
           width=1400,
           template='simple_white',
           color_continuous_scale=px.colors.sequential.Sunsetdark,
           hover_data=["Driver"]
        )

fig_top.update_layout(
    title="Counts By Final Driver Race Position All Time",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
                 tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
# fig_top.update_traces(marker_color='#246DF8')

In [15]:
fig_top.write_html("/Users/rdesh723/statpulse-html/plots/All Time/Counts By Driver Final Race Position.html",full_html=False, include_plotlyjs='cdn')

In [16]:
#By Grid Position

In [17]:
grouped_grid = GP.groupby(['FullName','GridPosition'])["GP"].agg('count').reset_index()
grouped_grid.columns=["Driver","GridPosition","Count"]
grouped_grid["Rank"] = grouped_grid.groupby(['GridPosition'])["Count"].rank(method="dense", ascending=False)
grouped_grid=grouped_grid.sort_values(by=["GridPosition","Count"],ascending=[True, False])

In [18]:
#Plot 1 lap at a time

In [19]:
grouped2_grid=grouped_grid[(grouped_grid['GridPosition'].isin([1]))&(grouped_grid['Count']>1)]

In [20]:
position=grouped2_grid["GridPosition"].iloc[0]

In [21]:
fig_lap_grid_pos=px.bar(grouped2_grid, 
           x="Driver", 
           y="Count", 
        #    facet_col='Position', facet_col_wrap=1, 
           height=800,
           width=1400,
           template='simple_white',
            color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 }
        )
fig_lap_grid_pos.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title="All Time #1 Driver Race Grid Start Position",
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_lap_grid_pos.update_traces(marker_color='#246DF8')

In [22]:
fig_lap_grid_pos.write_html("/Users/rdesh723/statpulse-html/plots/All Time/All Time Driver Grid 1 Start Position.html",full_html=False, include_plotlyjs='cdn')

In [23]:
#Plot the top grid positions

In [24]:
grouped_top_grid=grouped_grid[grouped_grid["Rank"]==1]
grouped_top_grid=grouped_top_grid[grouped_top_grid["GridPosition"]!=0]

In [25]:
fig_top_grid=px.bar(grouped_top_grid, 
           x="GridPosition", 
           y="Count", 
           color='Count',
           height=800,
           width=1400,
           template='simple_white',
           color_continuous_scale=px.colors.sequential.Teal,
           hover_data=["Driver"]
        )

fig_top_grid.update_layout(
    title="Counts By Driver Race Grid Start Position All Time",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
                 tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
# fig_top.update_traces(marker_color='#246DF8')

In [26]:
fig_top_grid.write_html("/Users/rdesh723/statpulse-html/plots/All Time/Counts By Driver Race Start Grid Position.html",full_html=False, include_plotlyjs='cdn')

In [27]:
#Driver Points All Time

In [28]:
Driver_all_time=GP.groupby(['FullName'])["Points"].agg('sum').reset_index().sort_values(by=["Points"],ascending=False)
Driver_filtered=Driver_all_time[(Driver_all_time['Points']>400)]

In [29]:
fig_driver_pts_all_time=px.histogram(Driver_filtered, 
           x="FullName", 
           y="Points", 
           color='FullName',
        #    text_auto=True,
           height=800,
           width=1400,
           template='simple_white',
           color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 },
            hover_data=["FullName"]
        )

fig_driver_pts_all_time.update_layout(
    title="Driver Points All Time",
    showlegend=False,
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=10)),
    font=dict(
        family="PT Sans Narrow",
        size=12,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_driver_pts_all_time.update_traces(marker_color='#246DF8')
fig_driver_pts_all_time.update_xaxes(title_text='')      # Remove axis titles
fig_driver_pts_all_time.update_yaxes(title_text='')
# fig_driver_pts_all_time.update_traces(textfont_size=12, textposition="outside")


In [30]:
fig_driver_pts_all_time.write_html("/Users/rdesh723/statpulse-html/plots/All Time/Driver Points All Time.html",full_html=False, include_plotlyjs='cdn')

In [31]:
#Qualifying

In [32]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.results_qualifying`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2_qualy = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')


Downloading: 100%|██████████|


In [33]:
GP_qualy=df_bq2_qualy
GP_qualy=GP_qualy.fillna(0)
GP_qualy["Position"]=GP_qualy["Position"].astype(int)
GP_qualy["GridPosition"]=GP_qualy["GridPosition"].astype(int)
GP_qualy["Points"]=GP_qualy["Points"].astype(int)
GP_qualy.head()

,DriverNumber,BroadcastName,Abbreviation,DriverId,TeamName,TeamColor,TeamId,FirstName,LastName,FullName,HeadshotUrl,CountryCode,Position,ClassifiedPosition,GridPosition,Q1,Q2,Q3,Time,Status,Points,Year,GP
0,10,P GASLY,GAS,gasly,Alpine,0093cc,alpine,Pierre,Gasly,Pierre Gasly,https://media.formula1.com/d_driver_fallback_i...,FRA,8,,0,0 days 00:01:17.149000,0 days 00:01:17.048000,0 days 00:01:16.892000,NaT,,0,2024,Mexico City Grand Prix
1,31,E OCON,OCO,ocon,Alpine,0093cc,alpine,Esteban,Ocon,Esteban Ocon,https://media.formula1.com/d_driver_fallback_i...,FRA,19,,0,0 days 00:01:17.617000,NaT,NaT,NaT,,0,2024,Mexico City Grand Prix
2,14,F ALONSO,ALO,alonso,Aston Martin,229971,aston_martin,Fernando,Alonso,Fernando Alonso,https://media.formula1.com/d_driver_fallback_i...,ESP,13,,0,0 days 00:01:17.307000,0 days 00:01:17.168000,NaT,NaT,,0,2024,Mexico City Grand Prix
3,18,L STROLL,STR,stroll,Aston Martin,229971,aston_martin,Lance,Stroll,Lance Stroll,https://media.formula1.com/d_driver_fallback_i...,CAN,14,,0,0 days 00:01:17.407000,0 days 00:01:17.294000,NaT,NaT,,0,2024,Mexico City Grand Prix
4,63,G RUSSELL,RUS,russell,Mercedes,27F4D2,mercedes,George,Russell,George Russell,https://media.formula1.com/d_driver_fallback_i...,GBR,5,,0,0 days 00:01:17.194000,0 days 00:01:16.937000,0 days 00:01:16.356000,NaT,,0,2024,Mexico City Grand Prix


In [34]:
#Get the counts by final position in Qualy

In [35]:
grouped_qualy = GP_qualy.groupby(['FullName','Position'])["GP"].agg('count').reset_index()
grouped_qualy.columns=["Driver","Position","Count"]
grouped_qualy["Rank"] = grouped_qualy.groupby(['Position'])["Count"].rank(method="dense", ascending=False)
grouped_qualy=grouped_qualy.sort_values(by=["Position","Count"],ascending=[True, False])

In [36]:
#Plot 1 lap at a time

In [37]:
grouped2_qualy=grouped_qualy[(grouped_qualy['Position'].isin([1]))&(grouped_qualy['Count']>0)]

In [38]:
position_qualy=grouped2_qualy["Position"].iloc[0]

In [39]:
fig_lap_qualy=px.bar(grouped2_qualy, 
           x="Driver", 
           y="Count", 
        #    facet_col='Position', facet_col_wrap=1, 
           height=800,
           width=1400,
           template='simple_white',
            color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 }
        )
fig_lap_qualy.update_layout(
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    title="Driver Qualifying Position #1 All Time",
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
                 tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_lap_qualy.update_traces(marker_color='#246DF8')

In [40]:
fig_lap_qualy.write_html("/Users/rdesh723/statpulse-html/plots/All Time/Driver Qualifying Results All Time.html",full_html=False, include_plotlyjs='cdn')

In [41]:
#Plot the top in each final position

In [42]:
grouped_top_qualy=grouped_qualy[grouped_qualy["Rank"]==1]
grouped_top_qualy=grouped_top[grouped_top["Position"]!=0]

In [43]:
fig_top_qualy=px.bar(grouped_top_qualy, 
           x="Position", 
           y="Count", 
           color='Count',
           height=800,
           width=1400,
           template='simple_white',
           color_continuous_scale=px.colors.sequential.Purp,
           hover_data=["Driver"]
        )

fig_top_qualy.update_layout(
    title="Counts By Qualifying Position All Time",
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15),
                 tickmode = 'linear',
    tick0 = 1,
    dtick = 1),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
# fig_top_qualy.update_traces(marker_color='#246DF8')

In [44]:
fig_top_qualy.write_html("/Users/rdesh723/statpulse-html/plots/All Time/Counts By Driver Qualifying Position All Time.html",full_html=False, include_plotlyjs='cdn')

In [45]:
#Driver Points for a Season

In [46]:
GP_2023.groupby(['Abbreviation'])["Points"].agg('sum').reset_index()

NameError: name 'GP_2023' is not defined

In [ ]:
fig_driver_pts=px.histogram(GP_2023.groupby(['FullName'])["Points"].agg('sum').reset_index().sort_values(by=["Points"],ascending=False), 
           x="FullName", 
           y="Points", 
           color='FullName',
           text_auto=True,
           template='simple_white',
           height=800,
           width=1200,
           color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 },
            hover_data=["FullName"]
        )

fig_driver_pts.update_layout(
    title="Driver Points",
    showlegend=False,
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_driver_pts.update_traces(marker_color='#246DF8')
# fig_driver_pts.update_traces(textfont_size=12, textposition="outside")


NameError: name 'GP_2023' is not defined

In [47]:
#Team Points for a Season

In [ ]:
fig_team_pts=px.histogram(GP_2023.groupby(['TeamName'])["Points"].agg('sum').reset_index().sort_values(by=["Points"],ascending=False), 
           x="TeamName", 
           y="Points", 
           color='TeamName',
        #    text_auto=True,
        template='simple_white',
           height=800,
           width=1200,
           color_discrete_map={
                 "Alpine": "#0093cc",
                 "Aston Martin": "#229971",
                 "Ferrari": "#E80020",
                 "Haas F1 Team": "#B6BABD",
                 "Kick Sauber": "#52e252",
                 "McLaren": "#FF8000",
                 "Mercedes": "#27F4D2",
                 "RB": "#6692FF",
                 "Red Bull Racing": "#3671C6",
                 "Williams": "#64C4FF" ,
                 "Alfa Romeo":"#C92D4B",
                 "AlphaTauri":"#5E8FAA",
                 "Racing Point":"#F596C8",
                 "Renault":"#FFF500",
                 "Toro Rosso":"#469bff",
                 "Force India":"#F596C8",
                 "Sauber":"#9B0000"
                 },
            hover_data=["TeamName"]
        )

fig_team_pts.update_layout(
    title="Team Points",
    showlegend=False,
    hoverlabel=dict(
        bgcolor="white",
        font_size=15,
        font_family="PT Sans Narrow"
    ),
    title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=15,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_team_pts.update_traces(textfont_size=12, textposition="outside")
